In [1]:
import mido
from mido import Message, MidiFile, MidiTrack,MetaMessage
import string
import numpy as np
import pandas as pd
from midi_array import *
import os
import csv


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# label path     
label_path = r'D:\BrownUnivercity\CS2470\final_proj\CS2470_final_project\data\label.csv'
# Define the folder path
folder_path = r'D:\BrownUnivercity\CS2470\final_proj\CS2470_final_project\data\test'
# load data
music,tag, align_length = get_music_data(folder_path, label_path)

In [4]:
# help function
def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    for i, label in enumerate(labels):
        one_hot_labels[i, label - 1] = 1
    return one_hot_labels

## CVAE

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, num_classes):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_size + num_classes, hidden_size)
        self.fc2_mean = nn.Linear(hidden_size, latent_size)
        self.fc2_logvar = nn.Linear(hidden_size, latent_size)
        
    def forward(self, x, y):
        # Concatenate input with class information
        x = torch.cat((x, y), dim=1)
        x = F.relu(self.fc1(x))
        mean = self.fc2_mean(x)
        logvar = self.fc2_logvar(x)
        return mean, logvar

class Decoder(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size, num_classes):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_size + num_classes, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, z, y):
        # Concatenate latent variable with class information
        z = torch.cat((z, y), dim=1)
        z = F.relu(self.fc1(z))
        reconstruction = torch.sigmoid(self.fc2(z))  # Use sigmoid for binary output
        return reconstruction

class CVAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, output_size, num_classes):
        super(CVAE, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, latent_size, num_classes)
        self.decoder = Decoder(latent_size, hidden_size, output_size, num_classes)
        
    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std
    
    def forward(self, x, y):
        x = x.to(torch.float32)
        y = y.to(torch.flost32)
        mean, logvar = self.encoder(x, y)
        z = self.reparameterize(mean, logvar)
        reconstruction = self.decoder(z, y)
        return reconstruction, mean, logvar



In [ ]:

# Define your dataset, dataloaders, loss function, optimizer, etc.
# Train your CVAE model using the defined components.
num_classes = 4
input_size = 10000 * 88
hidden_size = 256
latent_size = 32
output_size = 